In [ ]:
import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3

In [3]:
from sklearn.datasets import load_files
dataset = load_files('txt_sentoken', shuffle=False)

In [9]:
dataset.keys()

['target_names', 'data', 'target', 'DESCR', 'filenames']

In [32]:
dataset['target_names']

['neg', 'pos']

In [12]:
dataset['filenames'][0]
# dataset['data'][0]
# print(dataset['data'][0].decode('utf-8'))
dataset['target'][0]

0

In [14]:
import pandas as pd
data = pd.DataFrame({'data': dataset['data'], 'target': dataset['target']})

In [15]:
data.describe()

,target
count,2000.000000
mean,0.500000
std,0.500125
min,0.000000
25%,0.000000
50%,0.500000
75%,1.000000
max,1.000000


In [13]:
len(synopses)

100

## Stopwords, stemming, and tokenizing

In [21]:
# la primera vez hay que descargar la lista de 'stopwords': nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

In [22]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [28]:
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [43]:
# descargar diccionarios: nltk.download('punkt')
#  iterate over the list of synopses to create two vocabularies

totalvocab_stemmed = []
totalvocab_tokenized = []
for i in synopses:
    allwords_stemmed = tokenize_and_stem(i.decode('utf-8').strip()) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i.decode('utf-8').strip())
    totalvocab_tokenized.extend(allwords_tokenized)

In [106]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print('there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')
print(vocab_frame.head())

there are 199505 items in vocab_frame
          words
doctyp  doctype
html       html
html       html
lang=     lang=
en           en


In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [49]:
#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(synopses) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

CPU times: user 6.74 s, sys: 64 ms, total: 6.81 s
Wall time: 6.8 s
(100, 216)


In [ ]:
terms = tfidf_vectorizer.get_feature_names()

In [52]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

## K-Means Clustering

In [53]:
from sklearn.cluster import KMeans

num_clusters = 5

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

CPU times: user 60 ms, sys: 8 ms, total: 68 ms
Wall time: 137 ms


In [54]:
from sklearn.externals import joblib

joblib.dump(km,  'doc_cluster.pkl')

km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

In [57]:
with open("genres_list.txt") as file:
    genres = [line.strip() for line in file]

In [58]:
# Couldn't scrap ranks from IMDB
# Therefore, im creating random ranks
import random
ranks = []

for num in range(100):
    ranks.extend(random.sample(range(1, 10), 1))

In [59]:
films = { 'title': titles, 'rank': ranks, 'synopsis': synopses, 'cluster': clusters, 'genre': genres }

In [66]:
frame = pd.DataFrame(films, index = [clusters] , columns = ['rank', 'title', 'cluster', 'genre'])

ValueError: could not broadcast input array from shape (1039) into shape (1067)